In [1]:
"""
作者：librauee
微信公众号：老肥码码码
日期：2020.12.10
"""

'\n作者：librauee\n微信公众号：老肥码码码\n日期：2020.12.10\n'

In [13]:
import pandas as pd
pd.set_option('max_columns', 300)
from tqdm import tqdm
import warnings
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

warnings.filterwarnings('ignore')
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [3]:
# 目标编码
def kfold_mean(df_train, df_test, target, target_mean_list):
    folds = StratifiedKFold(n_splits=5)     #使用5折交叉验证.先获取了5折数据各自的索引

    mean_of_target = df_train[target].mean()

    for fold_, (trn_idx, val_idx) in tqdm(
            enumerate(folds.split(df_train, y=df_train['label']))):
        tr_x = df_train.iloc[trn_idx, :]
        vl_x = df_train.iloc[val_idx, :]

        for col in target_mean_list:
            df_train.loc[vl_x.index, f'{col}_target_enc'] = vl_x[col].map(
                tr_x.groupby(col)[target].mean())

    for col in target_mean_list:
        df_train[f'{col}_target_enc'].fillna(mean_of_target, inplace=True)

        df_test[f'{col}_target_enc'] = df_test[col].map(
            df_train.groupby(col)[f'{col}_target_enc'].mean())

        df_test[f'{col}_target_enc'].fillna(mean_of_target, inplace=True)
    return pd.concat([df_train, df_test], ignore_index=True)

feature_list =  ['HYZK', 'ZHIYE', 'ZHICHEN', 'ZHIWU', 'XUELI', 'DWJJLX', 'DWSSHY', 'GRZHZT'
                   ]
data = pd.concat([train, test], ignore_index=True)
data = kfold_mean(data[~data['label'].isna()], data[data['label'].isna()],
                  'label',
                  feature_list)

5it [00:00, 25.71it/s]


In [14]:
data.head(10)

,id,XINGBIE,CSNY,HYZK,ZHIYE,ZHICHEN,ZHIWU,XUELI,DWJJLX,DWSSHY,GRJCJS,GRZHZT,GRZHYE,GRZHSNJZYE,GRZHDNGJYE,GRYJCE,DWYJCE,DKFFE,DKYE,DKLL,label,HYZK_target_enc,ZHIYE_target_enc,ZHICHEN_target_enc,ZHIWU_target_enc,XUELI_target_enc,DWJJLX_target_enc,DWSSHY_target_enc,GRZHZT_target_enc,HYZK_COUNT,ZHIYE_COUNT,ZHICHEN_COUNT,ZHIWU_COUNT,XUELI_COUNT,DWJJLX_COUNT,DWSSHY_COUNT,GRZHZT_COUNT,GRYJCE-mean_gb_HYZK,GRYJCE-min_gb_HYZK,GRYJCE-max_gb_HYZK,GRYJCE/sum_gb_HYZK,DKFFE-mean_gb_HYZK,DKFFE-min_gb_HYZK,DKFFE-max_gb_HYZK,DKFFE/sum_gb_HYZK,DKLL-mean_gb_HYZK,DKLL-min_gb_HYZK,DKLL-max_gb_HYZK,DKLL/sum_gb_HYZK,DKYE-mean_gb_HYZK,DKYE-min_gb_HYZK,DKYE-max_gb_HYZK,DKYE/sum_gb_HYZK,GRJCJS-mean_gb_HYZK,GRJCJS-min_gb_HYZK,GRJCJS-max_gb_HYZK,GRJCJS/sum_gb_HYZK,GRZHSNJZYE-mean_gb_HYZK,GRZHSNJZYE-min_gb_HYZK,GRZHSNJZYE-max_gb_HYZK,GRZHSNJZYE/sum_gb_HYZK,GRZHDNGJYE-mean_gb_HYZK,GRZHDNGJYE-min_gb_HYZK,GRZHDNGJYE-max_gb_HYZK,GRZHDNGJYE/sum_gb_HYZK,GRYJCE-mean_gb_ZHIYE,GRYJCE-min_gb_ZHIYE,GRYJCE-max_gb_ZHIYE,GRYJCE/sum_gb_ZHIYE,DKFFE-mean_gb_ZHIYE,DKFFE-min_gb_ZHIYE,DKFFE-max_gb_ZHIYE,DKFFE/sum_gb_ZHIYE,DKLL-mean_gb_ZHIYE,DKLL-min_gb_ZHIYE,DKLL-max_gb_ZHIYE,DKLL/sum_gb_ZHIYE,DKYE-mean_gb_ZHIYE,DKYE-min_gb_ZHIYE,DKYE-max_gb_ZHIYE,DKYE/sum_gb_ZHIYE,GRJCJS-mean_gb_ZHIYE,GRJCJS-min_gb_ZHIYE,GRJCJS-max_gb_ZHIYE,GRJCJS/sum_gb_ZHIYE,GRZHSNJZYE-mean_gb_ZHIYE,GRZHSNJZYE-min_gb_ZHIYE,GRZHSNJZYE-max_gb_ZHIYE,GRZHSNJZYE/sum_gb_ZHIYE,GRZHDNGJYE-mean_gb_ZHIYE,GRZHDNGJYE-min_gb_ZHIYE,GRZHDNGJYE-max_gb_ZHIYE,GRZHDNGJYE/sum_gb_ZHIYE,GRYJCE-mean_gb_ZHICHEN,GRYJCE-min_gb_ZHICHEN,GRYJCE-max_gb_ZHICHEN,GRYJCE/sum_gb_ZHICHEN,DKFFE-mean_gb_ZHICHEN,DKFFE-min_gb_ZHICHEN,DKFFE-max_gb_ZHICHEN,DKFFE/sum_gb_ZHICHEN,DKLL-mean_gb_ZHICHEN,DKLL-min_gb_ZHICHEN,DKLL-max_gb_ZHICHEN,DKLL/sum_gb_ZHICHEN,DKYE-mean_gb_ZHICHEN,DKYE-min_gb_ZHICHEN,DKYE-max_gb_ZHICHEN,DKYE/sum_gb_ZHICHEN,GRJCJS-mean_gb_ZHICHEN,GRJCJS-min_gb_ZHICHEN,GRJCJS-max_gb_ZHICHEN,GRJCJS/sum_gb_ZHICHEN,GRZHSNJZYE-mean_gb_ZHICHEN,GRZHSNJZYE-min_gb_ZHICHEN,GRZHSNJZYE-max_gb_ZHICHEN,GRZHSNJZYE/sum_gb_ZHICHEN,GRZHDNGJYE-mean_gb_ZHICHEN,GRZHDNGJYE-min_gb_ZHICHEN,GRZHDNGJYE-max_gb_ZHICHEN,GRZHDNGJYE/sum_gb_ZHICHEN,GRYJCE-mean_gb_ZHIWU,GRYJCE-min_gb_ZHIWU,GRYJCE-max_gb_ZHIWU,GRYJCE/sum_gb_ZHIWU,DKFFE-mean_gb_ZHIWU,DKFFE-min_gb_ZHIWU,DKFFE-max_gb_ZHIWU,DKFFE/sum_gb_ZHIWU,DKLL-mean_gb_ZHIWU,DKLL-min_gb_ZHIWU,DKLL-max_gb_ZHIWU,DKLL/sum_gb_ZHIWU,DKYE-mean_gb_ZHIWU,DKYE-min_gb_ZHIWU,DKYE-max_gb_ZHIWU,DKYE/sum_gb_ZHIWU,GRJCJS-mean_gb_ZHIWU,GRJCJS-min_gb_ZHIWU,GRJCJS-max_gb_ZHIWU,GRJCJS/sum_gb_ZHIWU,GRZHSNJZYE-mean_gb_ZHIWU,GRZHSNJZYE-min_gb_ZHIWU,GRZHSNJZYE-max_gb_ZHIWU,GRZHSNJZYE/sum_gb_ZHIWU,GRZHDNGJYE-mean_gb_ZHIWU,GRZHDNGJYE-min_gb_ZHIWU,GRZHDNGJYE-max_gb_ZHIWU,GRZHDNGJYE/sum_gb_ZHIWU,GRYJCE-mean_gb_XUELI,GRYJCE-min_gb_XUELI,GRYJCE-max_gb_XUELI,GRYJCE/sum_gb_XUELI,DKFFE-mean_gb_XUELI,DKFFE-min_gb_XUELI,DKFFE-max_gb_XUELI,DKFFE/sum_gb_XUELI,DKLL-mean_gb_XUELI,DKLL-min_gb_XUELI,DKLL-max_gb_XUELI,DKLL/sum_gb_XUELI,DKYE-mean_gb_XUELI,DKYE-min_gb_XUELI,DKYE-max_gb_XUELI,DKYE/sum_gb_XUELI,GRJCJS-mean_gb_XUELI,GRJCJS-min_gb_XUELI,GRJCJS-max_gb_XUELI,GRJCJS/sum_gb_XUELI,GRZHSNJZYE-mean_gb_XUELI,GRZHSNJZYE-min_gb_XUELI,GRZHSNJZYE-max_gb_XUELI,GRZHSNJZYE/sum_gb_XUELI,GRZHDNGJYE-mean_gb_XUELI,GRZHDNGJYE-min_gb_XUELI,GRZHDNGJYE-max_gb_XUELI,GRZHDNGJYE/sum_gb_XUELI,GRYJCE-mean_gb_DWJJLX,GRYJCE-min_gb_DWJJLX,GRYJCE-max_gb_DWJJLX,GRYJCE/sum_gb_DWJJLX,DKFFE-mean_gb_DWJJLX,DKFFE-min_gb_DWJJLX,DKFFE-max_gb_DWJJLX,DKFFE/sum_gb_DWJJLX,DKLL-mean_gb_DWJJLX,DKLL-min_gb_DWJJLX,DKLL-max_gb_DWJJLX,DKLL/sum_gb_DWJJLX,DKYE-mean_gb_DWJJLX,DKYE-min_gb_DWJJLX,DKYE-max_gb_DWJJLX,DKYE/sum_gb_DWJJLX,GRJCJS-mean_gb_DWJJLX,GRJCJS-min_gb_DWJJLX,GRJCJS-max_gb_DWJJLX,GRJCJS/sum_gb_DWJJLX,GRZHSNJZYE-mean_gb_DWJJLX,GRZHSNJZYE-min_gb_DWJJLX,GRZHSNJZYE-max_gb_DWJJLX,GRZHSNJZYE/sum_gb_DWJJLX,GRZHDNGJYE-mean_gb_DWJJLX,GRZHDNGJYE-min_gb_DWJJLX,GRZHDNGJYE-max_gb_DWJJLX,GRZHDNGJYE/sum_gb_DWJJLX,GRYJCE-mean_gb_DWSSHY,GRYJCE-min_gb_DWSSHY,GRYJCE-max_gb_DWSSHY,GRYJCE/sum_gb_DWSSHY,DKFF

In [4]:
# 频数统计
cat_col = ['HYZK', 'ZHIYE', 'ZHICHEN', 'ZHIWU', 'XUELI', 'DWJJLX', 'DWSSHY', 'GRZHZT']
for col in cat_col:
    data[col + '_COUNT'] = data[col].map(data[col].value_counts())
    col_idx = data[col].value_counts()
    for idx in col_idx[col_idx < 10].index:
        data[col] = data[col].replace(idx, -1)  

In [5]:
# 偏离值特征
group_list = ['HYZK', 'ZHIYE', 'ZHICHEN', 'ZHIWU', 'XUELI', 'DWJJLX', 'DWSSHY', 'GRZHZT']
num_feature_list = ['GRYJCE', 'DKFFE', 'DKLL', 'DKYE', 'GRJCJS', 'GRZHSNJZYE', 'GRZHDNGJYE']                   
for group in group_list:
    for feature in num_feature_list:
        tmp = data.groupby(group)[feature].agg([sum, min, max, np.mean]).reset_index()
        tmp = pd.merge(data, tmp, on=group, how='left')
        data['{}-mean_gb_{}'.format(feature, group)] = data[feature] - tmp['mean']
        data['{}-min_gb_{}'.format(feature, group)] = data[feature] - tmp['min']
        data['{}-max_gb_{}'.format(feature, group)] = data[feature] - tmp['max']
        data['{}/sum_gb_{}'.format(feature, group)] = data[feature] / tmp['sum']  

In [16]:
data.shape

(55000, 261)

In [6]:
X_train, X_test = data[~data['label'].isna()], data[data['label'].isna()]
y = X_train['label']
drop_features = ['label', 'id', 'CSNY']

X_train = X_train.drop(drop_features, axis=1)
X_test = X_test.drop(drop_features, axis=1)

cat_col = ['HYZK', 'ZHIYE', 'ZHICHEN', 'ZHIWU', 'XUELI', 'DWJJLX', 'DWSSHY', 'GRZHZT']
X_train[cat_col] = X_train[cat_col].astype('category')
X_test[cat_col] = X_test[cat_col].astype('category')

In [7]:
# 评价指标
def tpr_weight_funtion(y_true,y_predict):

    d = pd.DataFrame()
    d['prob'] = list(y_predict)
    d['y'] = list(y_true)
    d = d.sort_values(['prob'], ascending=[0])
    y = d.y
    PosAll = pd.Series(y).value_counts()[1]
    NegAll = pd.Series(y).value_counts()[0]
    pCumsum = d['y'].cumsum()
    nCumsum = np.arange(len(y)) - pCumsum + 1
    pCumsumPer = pCumsum / PosAll
    nCumsumPer = nCumsum / NegAll
    TR1 = pCumsumPer[abs(nCumsumPer-0.001).idxmin()]
    TR2 = pCumsumPer[abs(nCumsumPer-0.005).idxmin()]
    TR3 = pCumsumPer[abs(nCumsumPer-0.01).idxmin()]
    return 0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3

In [10]:
# 训练&预测
KF = StratifiedKFold(n_splits=5, random_state=723)
params = {
    'verbose':-1, 
    'objective':'binary',
    'metric':'auc',
    'num_iterations': 10000, 
}


oof_lgb = np.zeros(len(X_train))
predictions_lgb = np.zeros((len(X_test)))

for fold_, (trn_idx, val_idx) in enumerate(KF.split(X_train.values, y.values)):
    print("fold n°{}".format(fold_))
    print('trn_idx:',trn_idx)
    print('val_idx:',val_idx)
    trn_data = lgb.Dataset(X_train.iloc[trn_idx],label=y.iloc[trn_idx])    
    val_data = lgb.Dataset(X_train.iloc[val_idx],label=y.iloc[val_idx])
    num_round = 10000
    clf = lgb.train(
        params,
                    trn_data,
                    num_round,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=500,
                    early_stopping_rounds=200,  
        categorical_feature=cat_col
    )
        
    oof_lgb[val_idx] = clf.predict(X_train.iloc[val_idx], num_iteration=clf.best_iteration)
    predictions_lgb[:] += clf.predict(X_test, num_iteration=clf.best_iteration) 
print("AUC score: {}".format(roc_auc_score(y, oof_lgb)))
print("TPR weight: {}".format(tpr_weight_funtion(y, oof_lgb)))

fold n°0
trn_idx: [ 7984  7985  7986 ... 39997 39998 39999]
val_idx: [   0    1    2 ... 8249 8282 8288]
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[173]	training's auc: 0.997885	valid_1's auc: 0.93858
fold n°1
trn_idx: [    0     1     2 ... 39997 39998 39999]
val_idx: [ 7984  7985  7986 ... 15997 15998 15999]
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[182]	training's auc: 0.99865	valid_1's auc: 0.933493
fold n°2
trn_idx: [    0     1     2 ... 39997 39998 39999]
val_idx: [16000 16001 16002 ... 24382 24393 24455]
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[153]	training's auc: 0.997841	valid_1's auc: 0.917636
fold n°3
trn_idx: [    0     1     2 ... 39997 39998 39999]
val_idx: [23964 23965 23966 ... 32011 32014 32019]
Training until validation scores don't improve for 200 rounds
[500]	training's auc: 1	valid_1's auc: 0.9183


In [11]:
submit = pd.read_csv('./data/submit.csv')
submit['label'] = predictions_lgb / 5
submit.to_csv('submit_baseline.csv', index=False)